# Hospital benchmarking using unsupervised learning methods

### Import python packages and a prepared dataset of hospital characteristics that include size, scope, patient population, and geography.

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
import altair as alt

In [2]:
continuous = pd.read_excel('..\\reference\\continuousFeatures.xlsx').sort_values('HOSP_NIS')  #deid hospital #
categorical = pd.read_excel('..\\reference\\categorical.xlsx').sort_values('HOSP_NIS')
#see feature list
print(list(continuous.drop(columns=['HOSP_NIS']).columns)+ \
      list(categorical.drop(columns=['HOSP_NIS']).columns))

['Cases', 'TransfersIn', 'EmergencyAdmit', '%<18', '%75+', '%NonWhite', '%MedicaidUI', 'SurgicalCases', 'AvgCharge', '%RoutineDischarge', 'AvgComorbidities', 'Region_EastNorthCentral', 'Region_EastSouthCentral', 'Region_MiddleAtlantic', 'Region_Mountain', 'Region_NewEngland', 'Region_Pacific', 'Region_SouthAtlantic', 'Region_WestNorthCentral', 'Region_WestSouthCentral', 'Area_BigMetro', 'Area_ExburbBigMetro', 'Area_MediumMetro', 'Area_Micro', 'Area_Rural', 'Area_SmallMetro', 'Area_Unknown']


### Normalize data, apply principal component analysis (PCA), and choose appropriate number of principal components

In [3]:
continuousNormalize = StandardScaler().fit_transform(continuous.drop(columns=['HOSP_NIS']))
featureArray = np.append(continuousNormalize,
                         categorical.drop(columns=['HOSP_NIS']).to_numpy(),axis=1)

pca = PCA().fit(featureArray)
pcaDf = pd.DataFrame({'pca': 0,'value':0}, index=[0]).append(
    pd.DataFrame({'pca':[x for x in range(1,np.cumsum(pca.explained_variance_ratio_).shape[0]+1)],
    'value': np.cumsum(pca.explained_variance_ratio_)})).reset_index(drop=True)

#find PCs to explain at least 90% of variance
cutOff = pcaDf.loc[pcaDf['value']>.9]['pca'].values[0]  
pointDf = pcaDf.loc[(pcaDf['pca']==cutOff)|(pcaDf['pca']==2)]
pointDf['label'] = np.round(pointDf['value']*100,1).astype(str) + '%'

#plot explained variance with Altair
point = alt.Chart(pointDf).mark_circle(color='white',stroke='#4c78a8',size=130, opacity=1
                                      ).encode(x='pca:Q',y='value:Q')

text = point.mark_text(fontSize=14,font='Arial',fontWeight='bold',color='#6e6e6e',dy=20,dx=20
                      ).encode(x='pca:Q',y='value:Q',text='label')

line = alt.Chart(pcaDf.loc[pcaDf['pca']<20]
        ).mark_line(size=5,color='#4c78a8',strokeCap='round').encode(
        x=alt.X('pca:Q',axis=alt.Axis(labelAngle=0,title='Number of components',tickMinStep=1,
                                      labelOffset=10)),
        y=alt.Y('value:Q',axis=alt.Axis(title='Cumulative explained variance',format='%',
                                        tickMinStep=.20,labelOffset=10),
        scale=alt.Scale(domain=(.03,1))))

(line+point+text).properties(height=350, width=500, 
                             title='Explained variance of principal components'
                    ).configure_axis(grid=False,labelColor='#6e6e6e',labelFont='Arial',
                                     labelFontSize=12,titleFont='Arial',titleFontSize=12,
                                     titleColor='#6e6e6e',domainWidth=0,tickSize=0
                    ).configure_view(strokeWidth=0).configure_title(fontSize=18,font='Arial',
                                     fontWeight='bold',color='#6e6e6e',anchor='start')

alt.LayerChart(...)

### Use a PCA biplot to explore relationships between continuous feature vectors

In [4]:
pca = PCA(n_components = 11).fit(featureArray)
X_pca = pca.transform(featureArray)

coeff = np.transpose(pca.components_[0:2, slice(0, 11, 1)])
biplotsDf = pd.DataFrame({'x':0,'y':0},index=[0]).append(
    pd.DataFrame({'x':coeff[:,0],'y':coeff[:,1]})).reset_index(drop=True)

cols = list(continuous.drop(columns=['HOSP_NIS']).columns)+ \
        list(categorical.drop(columns=['HOSP_NIS']).columns)
labels = cols[slice(0, 11, 1)]
labels.insert(0, '')
biplotsDf['label'] = labels

lines = {}
for i in biplotsDf.loc[biplotsDf.index!=0].index:
    lines[i] = alt.Chart(biplotsDf.loc[(biplotsDf.index==0)|(biplotsDf.index==i)]
                    ).mark_line(size=3,color='#805eba', 
                                point=alt.OverlayMarkDef(color='white',stroke='#805eba',size=60)
                    ).encode(x='x',  y='y', tooltip='label:N')
  

pcaScatterDf = pd.DataFrame({'PCA 1': X_pca[:,0], 'PCA 2':X_pca[:,1]})
pcaScatterDf['scalex'] = 1.1/(pcaScatterDf['PCA 1'].max()- pcaScatterDf['PCA 1'].min())
pcaScatterDf['scaley'] = 1.1/(pcaScatterDf['PCA 2'].max()- pcaScatterDf['PCA 2'].min())
pcaScatterDf['x'] = pcaScatterDf['PCA 1']*pcaScatterDf['scalex']
pcaScatterDf['y'] = pcaScatterDf['PCA 2']*pcaScatterDf['scaley']
pcaScatterDf['Hospital'] = continuous['HOSP_NIS']

pcaScatter = alt.Chart(pcaScatterDf).mark_point(size=15,filled=True,opacity=1, color='#dcdcdc'
                ).encode(
                x=alt.X('x:Q',axis=alt.Axis(title=None,labels=False),
                        scale=alt.Scale(domain=(biplotsDf['x'].min()-.05,biplotsDf['x'].max()+.05),clamp=True)),
                y=alt.Y('y:Q',axis=alt.Axis(title=None,labels=False),
                        scale=alt.Scale(domain=(biplotsDf['y'].min()-.05,biplotsDf['y'].max()+.05),clamp=True)),
                tooltip='Hospital')

(pcaScatter+lines[1]+lines[2]+lines[3]+lines[4]+lines[5]+lines[6]+lines[7]+ \
                 lines[8]+lines[9]+lines[10]+lines[11]
                ).properties(width=800, height=400, 
                             title=['PCA biplot - hover for feature names']
                ).configure_axis(grid=False,labelColor='#6e6e6e',labelFont='Arial',
                                 labelFontSize=12, titleFont='Arial',titleFontSize=12,
                                 titleColor='#6e6e6e',domainWidth=0,tickSize=0
                ).configure_view(strokeWidth=0
                ).configure_title(fontSize=18,font='Arial',fontWeight='bold',
                                  color='#6e6e6e',anchor='start')

alt.LayerChart(...)

# Method one: KMeans clusters

### find optimal number of clusters using the "elbow" method

In [5]:
sumSquares = []
for k in range(3,30):
    km = KMeans(n_clusters=k,random_state=42).fit(X_pca)
    sumSquares.append(km.inertia_)
    
getClusterSize = pd.DataFrame({'cluster':[i for i in range(3,30)], 'sumSquares':sumSquares})
pointLabelDf = getClusterSize.loc[getClusterSize['cluster']==10]
pointLabelDf['label'] = pointLabelDf['cluster'].astype(str) + ' clusters'

point = alt.Chart(pointLabelDf).mark_circle(color='white',stroke='#f58508',size=130, opacity=1
                                           ).encode(x='sumSquares:Q',y='cluster:Q')

text = point.mark_text(fontSize=14,font='Arial',fontWeight='bold',color='#6e6e6e',dy=-18,dx=35
                      ).encode(text='label')

line=alt.Chart(getClusterSize
              ).mark_line(size=5,color='#f58508',strokeCap='round').encode(
            x=alt.X('sumSquares:Q',axis=alt.Axis(labelAngle=0,title='Sum of squares distances',
                                                 tickMinStep=5000)),
            y=alt.Y('cluster:Q',axis=alt.Axis(title='Number of clusters')))

(line+point+text).properties(height=350, width=500, title = 'Find optimal number of clusters'
                    ).configure_axis(grid=False,labelColor='#6e6e6e',labelFont='Arial',
                                     labelFontSize=12,titleFont='Arial',titleFontSize=12,
                                     titleColor='#6e6e6e',domainWidth=0,tickSize=0
                    ).configure_view(strokeWidth=0
                    ).configure_title(fontSize=18,font='Arial',fontWeight='bold',
                                      color='#6e6e6e',anchor='start')

alt.LayerChart(...)

### Find and plot 10 KMeans clusters

In [6]:
kmeans = KMeans(n_clusters=10,random_state=42)
clusterAssignment = kmeans.fit_predict(X_pca)
clusterDf = pd.DataFrame(data=clusterAssignment.reshape(-1,1),columns=['Cluster'])
clusterDf['Hospital'] = continuous['HOSP_NIS']
X = PCA(n_components=2).fit_transform(featureArray)  #get x,y for viz
clusterDf['PC1'] = X[:,0]
clusterDf['PC2'] = X[:,1]

alt.Chart(clusterDf).mark_point(size=50,filled=True,opacity=.8,stroke='black',strokeWidth=.2
    ).encode(
    x=alt.X('PC1:Q',axis=alt.Axis(labels=False),scale=alt.Scale(domain=(-5,7),clamp=True)),
    y=alt.Y('PC2:Q',axis=alt.Axis(labels=False),scale=alt.Scale(domain=(-6,5),clamp=True)),
    color=alt.Color('Cluster:N',scale=alt.Scale(scheme='tableau10')),
    tooltip='Hospital:N'
    ).properties(width=750, height=500, title='Hospitals in ten clusters'
    ).configure_axis(grid=False,titleFont='Arial',titleFontSize=12,titleColor='#6e6e6e',
                     domainWidth=0,domainColor='#dcdcdc',tickSize=0
    ).configure_view(strokeWidth=0
    ).configure_title(fontSize=18,font='Arial',fontWeight='bold',color='#6e6e6e',anchor='start')

alt.Chart(...)

# Method two: Nearest Neighbors

### Find a benchmark group of size k for a focus hospital 

In [7]:
hospitalId=50436
k=50

cols = [f'PCA{i}' for i in range(X_pca.shape[1])]
featuresDf = pd.DataFrame(columns=cols, data=X_pca).fillna(0)
featuresDf['Hospital'] = continuous['HOSP_NIS']

#get focus hospital's array of feature values
knnX = featureArray[featuresDf.loc[featuresDf['Hospital'] == hospitalId].index[0]] 

neigh = NearestNeighbors(n_neighbors=k).fit(featureArray)
dist, ind = neigh.kneighbors(knnX.reshape(1, -1)) #input knnX array to find nearest neighbors

benchmarkGroup = []
for i in list(ind[0][1:]):
    benchmarkGroup.append(featuresDf.iloc[i]['Hospital'].astype(int))
        
print(benchmarkGroup)

[40565, 30595, 90082, 20085, 30649, 30681, 40334, 60141, 60307, 40495, 50025, 70575, 90406, 80150, 40582, 20003, 10071, 80138, 30699, 30527, 20303, 50062, 90164, 60397, 80040, 20190, 20394, 80010, 90110, 90298, 30049, 90525, 60210, 70655, 40072, 50574, 70437, 40445, 50126, 70220, 50141, 50363, 80124, 70072, 60044, 40436, 20177, 40579, 40656]


### Confirm results

In [8]:
seeBmDf = clusterDf.copy(deep=True)
seeBmDf['color'] = '#dcdcdc'
seeBmDf.loc[seeBmDf['Hospital'].isin([hospitalId]+benchmarkGroup),'color'] = '#e45756'

alt.Chart(seeBmDf).mark_point(size=50,filled=True,opacity=.8,stroke='black',strokeWidth=.2
    ).encode(
    x=alt.X('PC1:Q',axis=alt.Axis(labels=False),scale=alt.Scale(domain=(-5,7),clamp=True)),
    y=alt.Y('PC2:Q',axis=alt.Axis(labels=False),scale=alt.Scale(domain=(-6,5),clamp=True)),
    color=alt.Color('color:N',scale=None),
    tooltip='Hospital:N'
    ).properties(width=750, height=500, title='KNeighbors=50 benchmark group in red'
    ).configure_axis(grid=False,labelColor='#6e6e6e',labelFont='Arial',labelFontSize=12,
                titleFont='Arial',titleFontSize=12,titleColor='#6e6e6e',domainWidth=0,
                     domainColor='#dcdcdc',tickSize=0
    ).configure_view(strokeWidth=0
    ).configure_title(fontSize=18,font='Arial',fontWeight='bold',color='#6e6e6e',anchor='start')

alt.Chart(...)